run this on google collab

In [ ]:
#Install Required Packages
!pip install -qU langchain pinecone-client langchain-community sentence-transformers transformers

In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec


os.environ["PINECONE_API_KEY"] = "your_pinecone_api_key"
pinecone_api_key = "your_pinecone_api_key"
pc = Pinecone(api_key=pinecone_api_key)


In [2]:
index_name = "rag-qa-bot"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )


In [ ]:
from google.colab import files #add the data set or company info in text file
uploaded = files.upload()

Saving buisness_fax.txt to buisness_fax (13).txt


In [4]:
file_path = list(uploaded.keys())[0]

with open(file_path, 'r', encoding='utf-8') as f:
    raw_text = f.read()

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10
)

documents = text_splitter.create_documents([raw_text])


In [6]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


/tmp/ipython-input-6-3745702769.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public mod

In [7]:
from langchain_pinecone import PineconeVectorStore

vectorstore = PineconeVectorStore.from_documents(
    documents,
    embedding_model,
    index_name="rag-qa-bot"
)


In [8]:
retriever = vectorstore.as_retriever()

In [9]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA


qa_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",  # can replace with other models if needed
    tokenizer="google/flan-t5-base",
    max_length=512
)


llm = HuggingFacePipeline(pipeline=qa_pipeline)


Device set to use cpu
/tmp/ipython-input-9-3757149329.py:14: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=qa_pipeline)


In [10]:
retriever = vectorstore.as_retriever()

In [11]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
query = "my item is damaged what should i do?"#write your query
result = qa_chain(query)

print("Answer:", result["result"])

/tmp/ipython-input-12-1533879610.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain(query)


Answer: report it within 7 days
